# Bibliotecas

In [2]:
#Python Default 

import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

import pickle
import time
import random

#Essentials

import numpy as np
from bs4 import BeautifulSoup
import requests
#import func_timeout
import pandas as pd
#from itables import show
from scipy.optimize import curve_fit

#from objproxies import *

import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

#Qiskit

from qiskit.circuit import Gate
from qiskit.circuit.library import *
from qiskit import transpile
from qiskit.providers.models import *

#from qiskit_     import *
from qiskit.providers.fake_provider import *
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
import qiskit_aer

## Qiskit Nature
    
from qiskit_nature.units import *   #ANGSTROM

from qiskit_nature.second_q.drivers import *    #PySCFDriver

from qiskit_nature.second_q.mappers import * #JordanWignerMapper

from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult

from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF


## Qiskit Algorithms
    
from qiskit_algorithms.minimum_eigensolvers import VQE 
 
from qiskit_algorithms.optimizers import *    #SLSQP

from qiskit_algorithms.optimizers import L_BFGS_B

from qiskit.circuit.library import EfficientSU2   #EfficientSU2

## Qiskit Estimators

from qiskit.primitives import Estimator as Estimator_Nature

from qiskit_aer.primitives.estimator import Estimator as AerEstimator

#from qiskit_ibm_runtime import Estimator 

from qiskit_ibm_runtime import EstimatorV2 as Estimator

## Qiskit Noise Models

from qiskit_aer.noise import NoiseModel

from qiskit.providers.fake_provider import *

##Qiskit Runtime IBM

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

## Braket

from braket.tracking import Tracker
from qiskit_braket_provider import *



# Criando circuito para nosso problema de cálculo energético para molécula de hidrogênio - VQE

In [9]:
#Definindo estimator - backend da braket e Mapper Jordan Wigner
    
#from qiskit.primitives import Estimator

seed = 13686003

estimator = AerEstimator(
        backend_options={
            "method": "statevector",
            "coupling_map": None,
            "noise_model": None,
        },
        run_options={"seed": seed, "shots": 1024},
        transpile_options={"seed_transpiler": seed},
        approximation = False
        )

mapper = JordanWignerMapper()

optimizer = L_BFGS_B()



driver = PySCFDriver(
    atom= f"H 0 0 0; H 0 0 0.7",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

hproblem = driver.run()
#print(es_problem)



ansatz = UCCSD(
    hproblem.num_spatial_orbitals,
    hproblem.num_particles,
    mapper,
    initial_state=HartreeFock(
        hproblem.num_spatial_orbitals,
        hproblem.num_particles,
        mapper,
    ),
)

vqe = VQE(estimator, ansatz, optimizer)

vqe.initial_point = [0] * ansatz.num_parameters

calc = GroundStateEigensolver(mapper, vqe)

esr = calc.solve(hproblem) #electronic_structure_result 


# Modelo com a média de cada núméro de shots